### Route Templates

In [1]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge. Here is the question\n{input}'''

In [2]:
expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics. Here is the question\n{input}'''

In [3]:
# ADD YOUR OWN TEMPLATES !
empty_template = 'empty'

### Route Prompts

In [4]:
prompt_infos = [
    {'name':'empty','description':'Replies to empty questions','prompt_template':empty_template},
    {'name':'advanced physics','description': 'Answers advanced physics questions',
     'prompt_template':expert_template},
    {'name':'beginner physics','description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},
    
]

### ConversationChain

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # type: ignore

In [9]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    # chain = LLMChain(llm=llm, prompt=prompt)
    chain = prompt | llm
    destination_chains[name] = chain

In [161]:
# destination_chains

In [10]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

### Multi Routing Template

In [11]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

### Routing Destinations


In [13]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [14]:
print(destinations_str)

empty: Replies to empty questions
advanced physics: Answers advanced physics questions
beginner physics: Answers basic beginner physics questions


### Router Prompt

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [16]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [17]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
empty: Replies to empty questions
advanced physics: Answers advanced physics questions
beginner physi

### Routing Chain Call

In [18]:
from langchain.chains.router import MultiPromptChain

In [19]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [20]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

ValidationError: 3 validation errors for MultiPromptChain
destination_chains -> empty
  Can't instantiate abstract class Chain with abstract methods _call, input_keys, output_keys (type=type_error)
destination_chains -> advanced physics
  Can't instantiate abstract class Chain with abstract methods _call, input_keys, output_keys (type=type_error)
destination_chains -> beginner physics
  Can't instantiate abstract class Chain with abstract methods _call, input_keys, output_keys (type=type_error)

In [173]:
chain.run("How do magnets work?")



> Entering new  chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


"Great question! Let's dive into the fascinating world of magnets.\n\nMagnets are objects that have the amazing ability to attract certain materials, such as iron, nickel, or cobalt. But how do they do it? Well, it all comes down to something called magnetism.\n\nAt the atomic level, all materials are made up of tiny particles called atoms. Inside these atoms, you have even smaller particles called electrons, which have an electric charge. Now, here's where things get interesting.\n\nIn most materials, the electrons move around randomly, canceling out each other's magnetic fields. However, in certain materials, like iron, the electrons align themselves in the same direction, creating tiny regions called magnetic domains. These domains act like tiny magnets, with a north and south pole.\n\nNow, when you bring two magnets close together, something amazing happens. The magnetic domains in each magnet interact with each other. If you try to push the north poles of two magnets together, or 

In [174]:
chain.run("How do Feynman Diagrams work?")



> Entering new  chain...
advanced physics: {'input': 'How do Feynman Diagrams work?'}
> Finished chain.


'Feynman diagrams are graphical representations used in particle physics to visualize and calculate the interactions between elementary particles. They were developed by the physicist Richard Feynman as a powerful tool for understanding and calculating the behavior of subatomic particles based on quantum field theory.\n\nAt their core, Feynman diagrams depict the possible ways in which particles can interact and exchange energy and momentum. They provide a pictorial representation of the mathematical expressions that describe these interactions.\n\nIn a Feynman diagram, particles are represented by lines, with arrows indicating their direction of motion. The lines can be straight or wavy, representing different types of particles. For example, straight lines can represent fermions (such as electrons or quarks), while wavy lines can represent force-carrying bosons (such as photons or gluons).\n\nThe vertices where the lines meet in a Feynman diagram represent interactions between partic